## Registering new user with own credentials.

We don't have to use social media's to add new users to our DB. We can use our custom form. To do that let's first create a register page in our project. (src\app\(auth)\register).

In [ ]:
## page.tsx (RegisterPage)

import { registerNewUser } from "@/lib/actions"
import styles from "./register.module.css"

const RegisterPage = () => {
  return (
    <div className={styles.container}>
      <div className={styles.wrapper}>
        <form className={styles.form} action={registerNewUser}>
          <input type="text" placeholder="username" name="username"/>
          <input type="text" placeholder="email" name="email"/>
          <input type="text" placeholder="password" name="password"/>
          <input type="text" placeholder="password again" name="passwordRepeat"/>
          <button>Register</button>
        </form>
      </div>
    </div>
  )
}

export default RegisterPage

Now, we can add new function to our server actions file. (in lib forlder.

In [ ]:
## actions.js

#...

export const registerNewUser = async (formData) => {
    const {username, email, password, img, passwordRepeat} = Object.fromEntries(formData)

    # if the passwrod doesn't match the repeated password, return an error message
    if (password !== passwordRepeat) {
        return {error: "Passwords do not match!"}
    }

    try {
        connectToDb();  # connect to database

        const user = await User.findOne({username}) # try to find user with the passed username...

        if (user) { # if user already exists, return a message
            return {error: "User already exists" }
        }

        const userEmail = await User.findOne({email}) # check if email address exists in DB...

        if (userEmail) { # ... if yes, return a message
            return {error: "This email is already registered"}
        }

        # if the user doesn't exist in database, create a new user based on the passed values in the reigster form
        const newUser = new User({ 
            username,
            email,
            password,
            img,
        })
        
        await newUser.save(); # save new user
        console.log("new user saved to db")
        
        return {success: true} # we return a "success" (it will be important in chapter 14 - handling form errors)
        
    } catch (err) {  # handle error
        console.log(first)
        return {error: "Something went wrong"}
    }
}

Above solution is not perferct. Password field is a plain text, so it's not secure at all. We should first hash it and to do this, we can use different libraries. In our project we will use bcryptjs library. To use it, we have to install it first.

In [ ]:
npm install bcryptjs

Now import it in our server actions file and utilize it.

In [ ]:
## actions.js 

#...
import bcrypt from "bcryptjs";

#...

export const registerNewUser = async (formData) => {
    const {username, email, password, img, passwordRepeat} = Object.fromEntries(formData)

    # if the passwrod doesn't match the repeated password, return an error message
    if (password !== passwordRepeat) {
        return {error: "Passwords do not match!"}
    }

    try {
        connectToDb();  # connect to database

        const user = await User.findOne({username}) # try to find user with the passed username...

        if (user) { # if user already exists, return a message
            return {error: "User already exists" }
        }

        const userEmail = await User.findOne({email}) # check if email address exists in DB...

        if (userEmail) { # ... if yes, return a message
            return {error: "This email is already registered"}
        }
        
        const salt = await bcrypt.genSalt(10);  # create a salt variable
        const hashedPassword = await bcrypt.hash(password, salt) # hash password

        const newUser = new User({
            username,
            email,
            password: hashedPassword, # pass hashedPassword as a value of password
            img,
        })

        # if the user doesn't exist in database, create a new user based on the passed values in the reigster form
        const newUser = new User({ 
            username,
            email,
            password,
            img,
        })
        
        await newUser.save(); # save new user
        console.log("new user saved to db")
    } catch (err) {  # handle error
        console.log(first)
        return {error: "Something went wrong"}
    }
}

We have created a working registration form. Our next step is to create login form and authenticate the user. We will start with login form in our Login page.

In [ ]:
## page.tsx (LoginPage

import { handleGithubLogin, login } from '@/lib/actions';
import styles from "./login.module.css"

const LoginPage = async () => {

  return (
    <div className={styles.container}>
      <div className={styles.wrapper}>
        <form className={styles.form} action={handleGithubLogin}>
          <button>Login with GitHub</button>
        </form>
      </div>
      <div className={styles.wrapper}>
        <form className={styles.form} action={login}>
          <input type='text' placeholder='login' name='username' />
          <input type='password' placeholder='password' name='password' />
          <button>Login</button> 
        </form>
      </div>
    </div>
  )
}

export default LoginPage

Now, we have to add login function in our server actions file.

In [ ]:
## actions.js

#...

export const login = async (formData) => {
    const {username, password} = Object.fromEntries(formData) # we need username and password from the form

    try {
        await signIn("credentials", {username, password}); # we are using "credentials" as a provider (previously it was github)
        ## and we pass username and password
    } catch (error) {
        console.log(err)
        return {error: "Something went wrong"}
    }
}

Now we have to "tell" Auth.js how we encrypt the password. This is why we have to modify auth.js file.

In [ ]:
## auth.js

#... 
import Credentials from "next-auth/providers/credentials";  ## import Credentials provider
import bcrypt from "bcryptjs"; ## import bcrytpjs library

##### Create a function login #####

const login = async (credentials) => {
    try {
        connectToDb(); # try to connect with DB
        const user = await User.findOne({username: credentials.username}) # try to find the user in DB
        
        # if user was not found return information that the credentials are wrong
        if (!user) {
            throw new Error("Wrong credentials")
        }
        
        # if user is found in DB
        # check if the password is correct by using bcrypt compare method. We pass two arguments to compare, input 
        # credentials.pasword value and user password from DB.
        const isPasswordCorrect = await bcrypt.compare(credentials.password, user.password)
        
        # if the password is not correct
        if (!isPasswordCorrect) {
            throw new Error("Wrong credentails")
        }
        
        # if everything is OK. return user
        return user
    } catch (err) {
        console.log(err)
        thrown new Error("Failed to login")
    }
}

#### add new provider to NextAuth();

export const { handlers, signIn, signOut, auth } = NextAuth({
  providers: [
    GitHub({  
      clientId: process.env.AUTH_GITHUB_ID,
      clientSecret: process.env.AUTH_GITHUB_SECRET,
    }),
    ## add new provider here (Credentials)
    Credentials({
      async authorize(credentials) {
        try {
          const user = await login(credentials);  # we utilize login function we just created. We could do it here, but the code
            ## ... would be very long and illegible
          return user # if the authorization is ok, return the user
        } catch (error) {
          return null
        }
      }
    })
  ],
  callbacks: {
    async signIn({user, account, profile}) {
      if (account?.provider === "github") {
        connectToDb();
        try {
          const user = await User.findOne({email: profile?.email});
          if (!user) {
            const newUser = new User({
              username: profile?.login,
              email: profile?.email,
              img: profile?.avatar_url,
            });

            await newUser.save();
          }

        } catch (error) {
          console.log(error)
          return false
        }
      }
      return true
    },
  }
})